# YellowRose's super secret Colab quantization script
## 1) Download a KoboldAI compatible Repo GPTQ-for-LLaMA and setup folders

In [1]:
from google.colab import drive

drive.mount('/content/drive/')
import os
if not os.path.exists("/content/drive/My Drive/yr-gptq-colab"):
    !mkdir "/content/drive/My Drive/yr-gptq-colab"
%cd "/content/drive/My Drive/yr-gptq-colab"
if not os.path.exists("/content/drive/My Drive/yr-gptq-colab/base_models"):
    !mkdir "/content/drive/My Drive/yr-gptq-colab/base_models"
if not os.path.exists("/content/drive/My Drive/yr-gptq-colab/quantized_models"):
    !mkdir "/content/drive/My Drive/yr-gptq-colab/quantized_models"
if not os.path.exists("/content/drive/My Drive/yr-gptq-colab/GPTQ-for-LLaMa"):
    !git clone --single-branch --branch latestmerge https://github.com/YellowRoseCx/GPTQ-for-LLaMa.git
else:
      %cd GPTQ-for-LLaMa
      !git pull
%cd "/content/drive/My Drive/yr-gptq-colab/GPTQ-for-LLaMa"


## 2) Install GPTQ Requirements

In [2]:
!pip install -r requirements.txt

## 3) Use this space to git clone a basemodel if you want

In [ ]:
%cd "/content/drive/My Drive/yr-gptq-colab/base_models"
!git lfs install
!git clone https://huggingface.co/TehVenom/Pygmalion-7b-Merged-Safetensors



## 4) Select your 16 bit base model folder from your google drive "yr-gptq-colab/base_models/" folder

In [8]:
global basemodel_folder_path
global basemodel_folder_name
def select_folder():
    # get the root folder ID
    root_id = 'root'
    
    # get the list of folders in the root directory
    folder_list = []
    file_list = os.listdir('/content/drive/My Drive/yr-gptq-colab/base_models/')
    for file in file_list:
        if os.path.isdir('/content/drive/My Drive/yr-gptq-colab/base_models/' + file):
            folder_list.append(file)
    
    # print the list of folders
    print('Folders in your Google Drive:')
    if len(folder_list) >= 1:
      for i, folder in enumerate(folder_list):
        print(f'{i+1}: {folder}')
    else:
      print("There are no base model folders available")
    
    # prompt the user to select a folder
    while True:
        try:
            folder_num = int(input('Enter the number of the folder you want to use: '))
            basemodel_folder_name = folder_list[folder_num-1]
            basemodel_folder_path = os.path.join('/content/drive/My Drive/yr-gptq-colab/base_models/', basemodel_folder_name)
            print(f'Selected folder: {basemodel_folder_path}')
            print(f'Selected model: {basemodel_folder_name}')
            return basemodel_folder_path, basemodel_folder_name
        except:
            print('Invalid input. Please try again.')

# call the function to select a folder
basemodel_folder_path, basemodel_folder_name = select_folder()

## 5) Setup CUDA

In [4]:
%cd "/content/drive/My Drive/yr-gptq-colab/GPTQ-for-LLaMa"
!python ./setup_cuda.py install #NameError: name 'quant_cuda' is not defined



## 5.5) Login to Huggingface with a token generated from https://huggingface.co/settings/tokens

In [14]:
!huggingface-cli login

## 6) Quantize Model(s)


In [16]:
quant_command = (f"CUDA_VISIBLE_DEVICES=0 python llama.py \"{basemodel_folder_path}\" c4 --wbits 4 --save_safetensors \"/content/drive/My Drive/yr-gptq-colab/quantized_models/{basemodel_folder_name}-4bit.safetensors\"")
print(quant_command)
!{quant_command}

## 7) Eval Model(s)

In [22]:
eval_command = (f"CUDA_VISIBLE_DEVICES=0 python llama.py \"{basemodel_folder_path}\" c4 --wbits 4 --load \"/content/drive/My Drive/yr-gptq-colab/quantized_models/{basemodel_folder_name}-4bit.safetensors\" --eval --new-eval > results-{basemodel_folder_name}-eval.txt")
print(eval_command)
!{eval_command}